In [10]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 引入 parse 模組
import urllib.parse 

# 整理 json 使用的工具
import csv

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re

'''
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
#options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options )

#視窗大小設定 (寬，高)
driver.set_window_size(1200, 960)

# driver.maximize_window() #視窗最大化
# driver.minimize_window() #視窗最小化


'''
自訂變數
'''
# 放置爬取的資料
house_data = []
house_detail_data = []

# 放置 所有超連結
URLs_data = [] 

#放變數
j = 0

def openweb():
    #前往頁面
    driver.get("https://sale.591.com.tw/?shType=list&regionid=3");
    
    # 強制等待
    sleep(10)
    
    if len(driver.find_elements(By.CSS_SELECTOR, 'div.accreditPop') ) > 0:
        driver.find_element(By.CSS_SELECTOR, 'div>a.close').click()
    sleep(0.5)
    if len(driver.find_elements(By.CSS_SELECTOR, 'div.goodhouse-popbox-img') ) > 0:
        driver.find_element(By.CSS_SELECTOR, 'div.goodhouse-popbox-img').click()
    
# 分析頁面元素資訊
def parse():
    
    global j
    
    sleep(1)
    
    # 取得主要元素的集合
    house_list = driver.find_elements(By.CSS_SELECTOR, 'div.j-house.houseList-item.clearfix.z-hastag')
    
    # 逐一檢視元素
    for house_urls in house_list:
        # 取得連結
        a = house_urls.find_element(By.CSS_SELECTOR, 'div.houseList-item-main.oh > div.houseList-item-title > a')
        house_url = a.get_attribute('href')
#         print(house_url)
        
        URLs_data.append({
            "URLs":house_url
         })
        
    #爬六頁
    if j < 5:
        if len(driver.find_elements(By.CSS_SELECTOR, 'div.clearfix.relative > div.main-area > div.pages > a.pageNext') ) > 0:
            driver.find_element(By.CSS_SELECTOR, 'div.clearfix.relative > div.main-area > div.pages > a.pageNext').click()
            j+=1
            parse()

        else:
            visit_URLs()
#     print(URLs_data)
        
def visit_URLs():
    i=0
    for every_URLs in URLs_data:
        #讀取連結
        house_URL = every_URLs['URLs']
        #前往每個連結
        print(str(house_URL))
        driver.get(str(house_URL))
        sleep(1)
        
        get_data()
        
        house_data[i].extend(get_detail_data())
        i+=1
        print('有寫入詳細內容')    
    
def get_data():
    house_infos = driver.find_elements(By.CSS_SELECTOR, 'div#container > section.img-and-info.clearfix > div.info-box')
    for house_info in house_infos:
        total_price = house_info.find_element_by_class_name('info-price-num').text[0:4]
        print("總價:"+total_price+"萬")
        single_price = house_info.find_element_by_class_name('info-price-per').text
        print("單坪:"+single_price)
        
        house_value = driver.find_elements(By.CSS_SELECTOR,'div.info-floor-key')[0].text
        print("格局:"+house_value)
        house_age = driver.find_elements(By.CSS_SELECTOR,'div.info-floor-key')[1].text
        print("屋齡:"+house_age)
        house_area = driver.find_elements(By.CSS_SELECTOR,'div.info-floor-key')[2].text
        print("權狀坪數:"+house_area)
        house_floor = driver.find_elements(By.CSS_SELECTOR,'span.info-addr-value')[0].text
        print("樓層:"+house_floor)
        
        #判斷社區是否存在
        try: 
            if len(house_info.find_element_by_class_name('community-info-a').text) > 0:
                house_community = house_info.find_element_by_class_name('community-info-a').text
        except:
            house_community = 'Null'
        print("社區:"+house_community)
        
        house_addr = driver.find_elements(By.CSS_SELECTOR,'span.info-addr-value')[-1].text
        print("地址:"+house_addr)
            
        house_data.append([
            total_price,
            single_price,
            house_value,
            house_age,
            house_area,
            house_floor,
            house_community,
            house_addr
        ])
        print('有寫入house_data簡易內容')
    
        
        
        
def get_detail_data():    
    #詳細資訊
    house_detail_data = ['Null','Null','Null','Null','Null','Null','Null','Null','Null','Null','Null','Null']
    house_items = driver.find_elements(By.CSS_SELECTOR,'div.detail-house-item')
    sleep(0.5)
    for index, item in enumerate(house_items):
        try: 
            #if len(item.find_element_by_class_name('detail-house-key').text) > 0:
            house_detail_title = item.find_element_by_class_name('detail-house-key').text
            if house_detail_title == '現況':
                house_situation = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[0] = house_situation
                print('有抓到現況')
            elif house_detail_title == '型態':
                house_type = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[1] = house_type
                print('有抓到型態')
            elif house_detail_title == '裝潢程度':
                house_decorate = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[2] = house_decorate
                print('有抓到裝潢程度')
            elif house_detail_title == '管理費':
                house_manag_fee = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[3] = house_manag_fee
                print('有抓到管理費')
            elif house_detail_title == '帶租約':
                house_lease = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[4] = house_lease
                print('有抓到帶租約')
            elif house_detail_title == '法定用途':
                house_usage = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[5] = house_usage
                print('有抓到法定用途')
            elif house_detail_title == '車位':
                house_parking = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[6] = house_parking
                print('有抓到車位')
            elif house_detail_title == '公設比':
                house_amenities = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[7] = house_amenities
                print('有抓到公設比')
            elif house_detail_title == '主建物':
                house_main_building = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[8] = house_main_building
                print('有抓到主建物')
            elif house_detail_title == '共用部分':
                house_share = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[9] = house_share
                print('有抓到共用部分')
            elif house_detail_title == '附屬建物':
                house_attach_building = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[10] = house_attach_building
                print('有抓到附屬建物')
            elif house_detail_title == '土地坪數':
                house_ping = item.find_element_by_class_name('detail-house-value').text
                house_detail_data[11] = house_ping
                print('有抓到土地坪數')
            else:
                break
        except:
            break
#     house_detail_data.append([
#         house_situation,
#         house_type,
#         house_decorate,
#         house_manag_fee,
#         house_lease,
#         house_usage,
#         house_parking,
#         house_amenities,
#         house_main_building,
#         house_share,
#         house_attach_building,
#         house_ping
#     ])
    print('有寫入house_detail_data')
    print(house_detail_data)
    return house_detail_data
    

        
        
        
def createcsv():
    
    #print(house_detail_data)
    house_data.insert(0,['總價','單坪','格局','屋齡','權狀坪數','樓層','社區','地址','現況','型態','裝潢程度','管理費','帶租約','法定用途','車位','公設比','主建物','共用部分','附屬建物','土地坪數'])
    print(house_data)
    csvfile=open('housedata.csv','w', newline='')
    obj = csv.writer(csvfile)
    for row in house_data:
        obj.writerow(row)
    csvfile.close()            
                 
                 
openweb()
parse()
visit_URLs()
createcsv()

https://sale.591.com.tw/home/house/detail/2/7982993.html
總價:1450萬
單坪:46.44萬/坪
格局:3房2廳2衛2陽台
屋齡:32年
權狀坪數:31.22坪
樓層:3F/5F
社區:Null
地址:新北市新店區檳榔路19巷
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到法定用途
有抓到車位
有抓到公設比
有抓到主建物
有抓到共用部分
有抓到附屬建物
有抓到土地坪數
有寫入house_detail_data
['住宅', '公寓', '中檔裝潢', '無', '否', '住家用', '無', '9%', '26.19坪', '2.91坪', '2.89坪', '9.58坪']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7782451.html
總價:2800萬
單坪:52.35萬/坪
格局:3房2廳2衛2陽台
屋齡:1年
權狀坪數:53.49坪(含車位)
樓層:4F/18F
社區:麗寶ALL IN ONE
地址:新北市新莊區中央路556號
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到車位
有抓到公設比
有抓到主建物
有抓到共用部分
有抓到附屬建物
有抓到土地坪數
有寫入house_detail_data
['住宅', '電梯大樓', '高檔裝潢', '3925元', '否', 'Null', '13.2坪，平面式，已含售金內', '32%', '24.5坪', '13.08坪', '2.71坪', '5.75坪']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7989814.html
總價:2350萬
單坪:1萬/坪
格局:3房2廳2衛
屋齡:12年
權狀坪數:2350坪(含車位)
樓層:5F/12F
社區:東禧一品
地址:新北市三重區三信路
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到法定用途
有抓到公設比
有抓到主建物
有抓到共用部分
有抓到附屬建物
有抓到土地坪數
有寫入house_det

有寫入house_detail_data
['住宅', '電梯大樓', '簡易裝潢', '2350元', '否', '住家用', '5.\n\n4\n\n7坪，平面式，已含售金內', 'Null', '40.51坪', 'Null', '6.46坪', '7.74坪']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7969643.html
總價:998萬
單坪:17.71萬/坪
格局:4房3廳3衛2陽台
屋齡:30年
權狀坪數:56.35坪(含車位)
樓層:2F/2F
社區:黎明清境
地址:新北市新店區黎明路52巷
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到法定用途
有抓到車位
有抓到主建物
有抓到附屬建物
有抓到土地坪數
有寫入house_detail_data
['住宅', '別墅', '簡易裝潢', '1800元', '否', '住家用', '9.24坪，平面式，已含售金內', 'Null', '39.62坪', 'Null', '7.49坪', '25.4坪']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7948995.html
總價:1950萬
單坪:56.31萬/坪
格局:3房2廳2衛1陽台
屋齡:6年
權狀坪數:34.63坪
樓層:2F/14F
社區:玉上園
地址:新北市新店區寶橋路78巷
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到帶租約
有抓到法定用途
有抓到車位
有抓到公設比
有抓到主建物
有抓到共用部分
有抓到附屬建物
有寫入house_detail_data
['住宅', '電梯大樓', '高檔裝潢', 'Null', '否', '住家用', '無', '30%', '21.11坪', '10.28坪', '3.24坪', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7946034.html
總價:1560萬
單坪:29.88萬/坪
格局:4房2廳2衛2陽台
屋齡:14年
權狀坪數:59.46坪(含車位)
樓層:整棟/3F
社區:Null
地址

有抓到車位
有寫入house_detail_data
['住宅', '電梯大樓', '高檔裝潢', '500元', '否', '商業用', '無', 'Null', 'Null', 'Null', 'Null', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7985781.html
總價:998萬
單坪:80.74萬/坪
格局:1房1廳1衛
屋齡:10年
權狀坪數:12.36坪
樓層:31F/40F
社區:新巨蛋
地址:新北市板橋區文化路一段360號
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到法定用途
有抓到車位
有寫入house_detail_data
['套房', '電梯大樓', '簡易裝潢', '856元', '否', '住家用', '無', 'Null', 'Null', 'Null', 'Null', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7956998.html
總價:1798萬
單坪:37.08萬/坪
格局:3房2廳2衛2陽台
屋齡:9年
權狀坪數:48.49坪(含車位)
樓層:13F/15F
社區:捷運躍境
地址:新北市新莊區中正路893巷
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到法定用途
有抓到車位
有寫入house_detail_data
['住宅', '電梯大樓', '中檔裝潢', '2319元', '否', '集合住宅', '8.1坪，平面式，已含售金內', 'Null', 'Null', 'Null', 'Null', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7981428.html
總價:1400萬
單坪:62.17萬/坪
格局:1房1廳1衛
屋齡:10年
權狀坪數:22.52坪(含車位)
樓層:28F/40F
社區:新巨蛋
地址:新北市板橋區文化路一段
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到法定用

總價:1325萬
單坪:51.07萬/坪
格局:4房2廳2衛2陽台
屋齡:22年
權狀坪數:25.95坪
樓層:11F/11F
社區:大阪城紅寶石
地址:新北市板橋區四川路二段245巷
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到帶租約
有抓到法定用途
有抓到車位
有寫入house_detail_data
['住宅', '電梯大樓', '簡易裝潢', 'Null', '否', '住家用', '無', 'Null', 'Null', 'Null', 'Null', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7932158.html
總價:888萬
單坪:49.31萬/坪
格局:1房1廳1衛1陽台
屋齡:3年
權狀坪數:18.01坪
樓層:2F/10F
社區:成功華廈
地址:新北市三重區成功路
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到法定用途
有抓到車位
有抓到公設比
有抓到主建物
有抓到共用部分
有抓到附屬建物
有抓到土地坪數
有寫入house_detail_data
['住宅', '電梯大樓', '簡易裝潢', '1380元', '否', '住家用', '無', '42%', '9.11坪', '7.51坪', '1.39坪', '2.25坪']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7940418.html
總價:998萬
單坪:36萬/坪
格局:2房1廳1衛2陽台
屋齡:12年
權狀坪數:32.97坪(含車位)
樓層:5F/10F
社區:台北寶成二期
地址:新北市泰山區仁武街139巷
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到帶租約
有抓到法定用途
有抓到車位
有抓到公設比
有抓到主建物
有抓到共用部分
有抓到附屬建物
有寫入house_detail_data
['住宅', '電梯大樓', '簡易裝潢', 'Null', '否', '住家用', '8.58坪，機械式，已含售金內', '28%', '14.79坪', '6.84坪', '2.76坪', 'Null']
有寫入詳

有抓到現況
有抓到型態
有抓到裝潢程度
有抓到帶租約
有抓到法定用途
有抓到車位
有寫入house_detail_data
['住宅', '公寓', '簡易裝潢', 'Null', '否', '住家用', '無', 'Null', 'Null', 'Null', 'Null', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7947020.html
總價:3450萬
單坪:89.31萬/坪
格局:3房2廳2衛
屋齡:5年
權狀坪數:38.63坪(含車位)
樓層:5F/15F
社區:三輝都匯2部曲
地址:新北市板橋區莊敬路
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到帶租約
有抓到法定用途
有寫入house_detail_data
['住宅', '電梯大樓', '高檔裝潢', 'Null', '否', '集合住宅', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7967646.html
總價:5888萬
單坪:62.75萬/坪
格局:4房2廳3衛
屋齡:1年
權狀坪數:93.83坪(含車位)
樓層:16F/17F
社區:Null
地址:新北市新店區民族路
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到法定用途
有抓到車位
有抓到公設比
有抓到主建物
有抓到共用部分
有抓到附屬建物
有寫入house_detail_data
['住宅', '電梯大樓', '尚未裝潢', '6668元', '否', '住家用', '23坪，平面式，已含售金內', '31%', '43.3坪', '21.65坪', '5.88坪', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7992951.html
總價:858萬
單坪:22.96萬/坪
格局:2房2廳1衛1陽台
屋齡:12年
權狀坪數:37.37坪(含車位)
樓層:6F/8F
社區:綠灣
地址:新北市新店區安祥路
有寫入house_da

總價:2550萬
單坪:55.74萬/坪
格局:4房2廳2衛2陽台
屋齡:20年
權狀坪數:45.75坪(含車位)
樓層:4F/8F
社區:Null
地址:新北市新店區中央四街
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到帶租約
有抓到法定用途
有抓到公設比
有抓到主建物
有抓到共用部分
有抓到附屬建物
有寫入house_detail_data
['住宅', '電梯大樓', '簡易裝潢', 'Null', '否', '住家用', 'Null', '30%', '28.819坪', '13.67坪', '3.263坪', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7962109.html
總價:1498萬
單坪:46.04萬/坪
格局:3房2廳2衛2陽台
屋齡:29年
權狀坪數:32.54坪
樓層:5F/5F
社區:Null
地址:新北市新店區文化路80巷
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到帶租約
有抓到法定用途
有抓到車位
有抓到主建物
有抓到附屬建物
有寫入house_detail_data
['住宅', '公寓', '高檔裝潢', 'Null', '否', '住家用', '無', 'Null', '30.55坪', 'Null', '1.97坪', 'Null']
有寫入詳細內容
https://sale.591.com.tw/home/house/detail/2/7827003.html
總價:1780萬
單坪:42.53萬/坪
格局:3房2廳2衛1陽台
屋齡:1年
權狀坪數:41.85坪(含車位)
樓層:4F/15F
社區:Null
地址:新北市板橋區大漢街
有寫入house_data簡易內容
有抓到現況
有抓到型態
有抓到裝潢程度
有抓到管理費
有抓到帶租約
有抓到法定用途
有抓到車位
有寫入house_detail_data
['住宅', '電梯大樓', '簡易裝潢', '2088元', '否', '住家用', '9.34坪，平面式，已含售金內', 'Null', 'Null', 'Null', 'Null', 'Null']
有寫入詳細內容
https://sale.591.com.tw/